In [1]:
# ======================================================================================================
# PROBLEM A3 
#
# Build a classifier for the Human or Horse Dataset with Transfer Learning. 
# The test will expect it to classify binary classes.
# Note that all the layers in the pre-trained model are non-trainable.
# Do not use lambda layers in your model.
#
# The horse-or-human dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
# Inception_v3, pre-trained model used in this problem is developed by Google.
#
# Desired accuracy and validation_accuracy > 93%.
# =======================================================================================================

import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [2]:
def solution_A3():
    inceptionv3='https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    urllib.request.urlretrieve(inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
    local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

    pre_trained_model = InceptionV3( input_shape = (150,150,3),
                                    include_top = False,
                                    weights = None)
    pre_trained_model.load_weights(local_weights_file)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer("mixed9")
    last_output = last_layer.output

    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    local_file = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/horse-or-human')

    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    local_file = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/validation-horse-or-human')
    zip_ref.close()

    
    x = layers.Flatten()(last_output)
    x = layers.Dense(1028,activation = "relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(10,activation = "relu")(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)

    model.compile(optimizer=RMSprop(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['acc'])

    train_dir = 'data/horse-or-human'
    validation_dir = 'data/validation-horse-or-human'

    train_datagen = ImageDataGenerator(
       rescale = 1./255,
       width_shift_range = 0.2,
       zoom_range = 0.2,
       rotation_range = 40,
       horizontal_flip = True,
       vertical_flip = True)
    
    val_datagen = ImageDataGenerator(
        rescale = 1./255
    )


    train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size = (150,150),
                                                        batch_size = 20,
                                                        class_mode = "binary")
    validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                             target_size = (150,150),
                                                             batch_size = 20,
                                                             class_mode = "binary")
    model.fit(train_generator,
              steps_per_epoch = 16,
              epochs = 5,
              validation_data = validation_generator,
              validation_steps = 8 )
    

    return model

In [3]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    model = solution_A3()
    model.save("model_A3.h5")


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/5
16/16 [==============================] - 29s 2s/step - loss: 0.2311 - acc: 0.8844 - val_loss: 0.0924 - val_acc: 0.9625
Epoch 2/5
16/16 [==============================] - 24s 2s/step - loss: 0.0878 - acc: 0.9750 - val_loss: 0.1295 - val_acc: 0.9438
Epoch 3/5
16/16 [==============================] - 23s 1s/step - loss: 0.0277 - acc: 0.9875 - val_loss: 0.1553 - val_acc: 0.9438
Epoch 4/5
16/16 [==============================] - 22s 1s/step - loss: 0.0408 - acc: 0.9837 - val_loss: 0.0352 - val_acc: 0.9812
Epoch 5/5
16/16 [==============================] - 23s 1s/step - loss: 0.0259 - acc: 0.9906 - val_loss: 0.0709 - val_acc: 0.9750
